# Getting Started with using Quest.

In [ ]:
from pprint import pprint
import itertools
from IPython.display import display, Markdown

import param
import ipywidgets as widgets
from paramnb import Widgets 

import quest
print('\nQUEST version %s' % quest.api.get_quest_version())
print('\nQUEST API version %s' % quest.api.get_api_version())

### Quest Provider Plugins:

In [ ]:
providers = quest.api.get_providers()
print("Providers: ")
pprint(providers)

services = quest.api.get_services()
print("\nServices: ")
pprint(services)

publishers = quest.api.get_publishers()
print("\nPublishers: ")
pprint(publishers)

### Quest Tool Plugins:

In [ ]:
tools = quest.api.get_tools()
pprint(tools)

### Quest Projects

In [ ]:
print('\033[1m' + 'Showing existing projects.' + '\033[0m')
projects = quest.api.get_projects()
pprint(projects)

print('\n---------------------------\n')
print('\033[1m' + 'Creating a new project.' + '\033[0m')

if 'quest-demo' in projects:
    quest.api.delete_project('quest-demo')

quest.api.new_project('Quest-Demo', 'Quest-Demo', 'For demostrating how to use Quest.')

### Quest Collections

In [ ]:
print('\033[1m' + 'Showing existing collections.' + '\033[0m')
collections = quest.api.get_collections()
pprint(collections)

print('\n---------------------------\n')
print('\033[1m' + 'Creating a new project.' + '\033[0m')

collection_name = 'col-test'
if collection_name in collections:
    quest.api.delete(collection_name)

quest.api.new_collection(collection_name, collection_name, 'For demonstrating how to add collections to Quest.')

### Getting Catalog Entries:

In [ ]:
filters = {}
get_parameter = widgets.Dropdown(options=quest.api.get_mapped_parameters(), description='Parameters:')
display(get_parameter)

In [ ]:
if get_parameter.value != '':
    filters['parameter'] = get_parameter.value
    get_service = widgets.Dropdown(options=quest.api.get_services(parameter=get_parameter.value), description='Services:')
else:
    get_service = widgets.Dropdown(options=quest.api.get_services(), description='Services:')
display(get_service)

In [ ]:
get_geom = widgets.Dropdown(options=['', 'Point', 'Polygon', 'Line'], description='Gemoetry:')
display(get_geom)

In [ ]:
if get_geom.value != '':
    filters['geom_type'] = get_geom.value
    
long_min = widgets.Text(description='long min:')
lat_min = widgets.Text(description='lat min:')
long_max = widgets.Text(description='long max:')
lat_max = widgets.Text(description='lat max:')
display(long_min, lat_min, long_max, lat_max)

In [ ]:
value = len(long_min.value) * len(lat_min.value) * len(long_max.value) * len(lat_max.value)
if value != 0:
    bbox = [long_min.value, lat_min.value, long_max.value, lat_max.value]
    filters['bbox'] = bbox

In [ ]:
cache = widgets.Checkbox(value=False, description='Update Cache')
display(cache)

In [ ]:
filters = {'parameter': get_parameter.value}
datasets = quest.api.search_catalog(uris=get_service.value, filters=filters, update_cache=cache.value, as_dataframe=True)
datasets

In [ ]:
get_datasets = widgets.Dropdown(options=datasets.index.tolist()[:50], description='Datasets:')
display(get_datasets)

### Add the dataset to a collection:

In [ ]:
catalog_entry = quest.api.add_datasets(collection_name, get_datasets.value)
catalog_entry

### Get the download options:

In [ ]:
download_options = quest.api.get_download_options(catalog_entry, fmt='param')[catalog_entry[0]]
Widgets(download_options)

### Download the selected data:

In [ ]:
staged_id = quest.api.stage_for_download(catalog_entry, options=download_options)
print('\033[1m' + "Staged ID: " + '\033[0m' + staged_id[0])

quest.api.download_datasets(staged_id)